## Test Data Lake - project 4

In [26]:
import os
import configparser
from pyspark.sql import SparkSession
import pandas as pd

# Make sure that your AWS credentials are loaded as env vars

In [27]:
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['ROLE']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['ROLE']['AWS_SECRET_ACCESS_KEY']

## Create spark session with hadoop-aws package

In [3]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark
spark = create_spark_session()

## Load S3 data

In [28]:
input_path = config['DATA']['OUTPUT_PATH']
songs_path = os.path.join(input_path, 'songs.parquet')
user_path = os.path.join(input_path, 'user.parquet')
time_path = os.path.join(input_path, 'time.parquet')
songplays_path = os.path.join(input_path, 'songplays.parquet')
artist_path = os.path.join(input_path, 'artist.parquet')

### load song data

In [40]:
df_song = spark.read.parquet(songs_path)
df_song.printSchema()
df_song.toPandas().head()

root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- artist_id: string (nullable = true)



,song_id,title,duration,year,artist_id
0,SOKTJDS12AF72A25E5,Drown In My Own Tears (24-Bit Digitally Remast...,192.52200,0,ARA23XO1187B9AF18F
1,SOEKAZG12AB018837E,I'll Slap Your Face (Entertainment USA Theme),129.85424,2001,ARSVTNL1187B992A91
2,SOAFBCP12A8C13CC7D,King Of Scurf (2007 Digital Remaster),301.40036,1972,ARTC1LV1187B9A4858
3,SORRNOC12AB017F52B,The Last Beat Of My Heart (b-side),337.81506,2004,ARSZ7L31187FB4E610
4,SOQPWCR12A6D4FB2A3,A Poor Recipe For Civic Cohesion,118.07302,2005,AR73AIO1187B9AD57B


### Total songs created grouped by years

In [52]:
df_song.createOrReplaceTempView("song")
spark.sql("""
    SELECT year, count(song_id) as num_songs
    FROM song
    GROUP BY year
    ORDER BY num_songs DESC
""").show(5)

,year,num_songs
0,0,9
1,2004,3
2,2007,1
3,2006,1
4,1978,1


### load user data

In [41]:
df_user = spark.read.parquet(user_path)
df_user.printSchema()
df_user.toPandas().head()

root
 |-- userId: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- level: string (nullable = true)



,userId,firstName,lastName,gender,level
0,60,Devin,Larson,M,free
1,68,Jordan,Rodriguez,F,free
2,90,Andrea,Butler,F,free
3,77,Magdalene,Herman,F,free
4,89,Kynnedi,Sanchez,F,free


### Check which group have more free account

In [55]:
df_user.createOrReplaceTempView("users")
spark.sql("""
    SELECT count(userId) as total_users, gender
    FROM users
    WHERE level='free'
    GROUP BY gender
    ORDER BY total_users DESC
""").show()

,total_useres,gender
0,40,F
1,35,M


### load time data

In [42]:
df_time = spark.read.parquet(time_path)
df_time.printSchema()
df_time.toPandas().head()

root
 |-- timestamp: timestamp (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- weekday: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)



,timestamp,hour,day,week,weekday,year,month
0,2018-11-15 00:30:26.796,0,15,46,5,2018,11
1,2018-11-15 00:41:21.796,0,15,46,5,2018,11
2,2018-11-15 00:45:41.796,0,15,46,5,2018,11
3,2018-11-15 03:44:09.796,3,15,46,5,2018,11
4,2018-11-15 05:48:55.796,5,15,46,5,2018,11


### load songplays data

In [43]:
df_songplays = spark.read.parquet(songplays_path)
df_songplays.printSchema()
df_songplays.toPandas().head()

root
 |-- ts: timestamp (nullable = true)
 |-- userId: string (nullable = true)
 |-- level: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- location: string (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- songplay_id: long (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)



,ts,userId,level,song_id,artist_id,sessionId,location,userAgent,songplay_id,month,year
0,2018-11-15 16:19:05.796,97,paid,SOBLFFE12AF72AA5BA,ARJNIUY12298900C91,605,"Lansing-East Lansing, MI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",0,11,2018


### Which users listened a lot of music?

In [57]:
df_songplays.createOrReplaceTempView("songplays")
spark.sql("""
    SELECT userId, count(sessionId) as total_session
    FROM songplays
    GROUP by userId
    order by total_session desc
""").show(5)

+------+-------------+
|userId|total_session|
+------+-------------+
|    97|            1|
+------+-------------+



### load artist data

In [44]:
df_artist = spark.read.parquet(artist_path)
df_artist.printSchema()
df_artist.toPandas().head()

root
 |-- artist_id: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_longitude: double (nullable = true)



,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARTC1LV1187B9A4858,The Bonzo Dog Band,"Goldsmith's College, Lewisham, Lo",51.45360,-0.01802
1,ARA23XO1187B9AF18F,The Smithereens,"Carteret, New Jersey",40.57885,-74.21956
2,ARSVTNL1187B992A91,Jonathan King,"London, England",51.50632,-0.12714
3,AR73AIO1187B9AD57B,Western Addiction,"San Francisco, CA",37.77916,-122.42005
4,ARXQBR11187B98A2CC,Frankie Goes To Hollywood,"Liverpool, England",NaN,NaN


### Find All Artists from England

In [62]:
df_artist.createOrReplaceTempView("artists")
spark.sql("""
    SELECT artist_id, artist_name, artist_location
    FROM artists
    WHERE artist_location 
    LIKE '%England%'
""").show(5)

+------------------+--------------------+------------------+
|         artist_id|         artist_name|   artist_location|
+------------------+--------------------+------------------+
|ARSVTNL1187B992A91|       Jonathan King|   London, England|
|ARXQBR11187B98A2CC|Frankie Goes To H...|Liverpool, England|
+------------------+--------------------+------------------+

